In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
from torchinfo import summary
import pandas as pd
import os

In [ ]:
data_path = './data/spectrograms' #looking in subfolder train
dataset = datasets.ImageFolder(root=data_path,transform=transforms.Compose([transforms.Grayscale(),transforms.ToTensor()]))
class_map=dataset.class_to_idx
print(class_map)


In [ ]:
#split data to test and train
#use 80% to train
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

print("Training size:", len(train_dataset))
print("Testing size:",len(test_dataset))

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=15,
    num_workers=2,
    shuffle=True
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=50,
    num_workers=2,
    shuffle=True
)
print(torch.cuda.is_available())

In [ ]:
class CNNet (nn.Module):
    # ----------------------------
    # Build the model architecture
    # ----------------------------
    def __init__(self):
        super().__init__()
        conv_layers = []

        # First Convolution Block with Relu and Batch Norm. Use Kaiming Initialization
        self.conv1 = nn.Conv2d(1, 8, kernel_size=5)
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        nn.init.kaiming_normal_(self.conv1.weight, a=0.1)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1,self.bn1]

        # Second Convolution Block
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3)
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        nn.init.kaiming_normal_(self.conv2.weight, a=0.1)
        self.conv2.bias.data.zero_()
        self.drop2 = nn.Dropout2d()
        self.pool2=nn.MaxPool2d(2)
        conv_layers += [self.conv2, self.pool2, self.relu2,self.bn2]

        # third Convolution Block
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3)
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        nn.init.kaiming_normal_(self.conv3.weight, a=0.1)
        self.conv3.bias.data.zero_()
        self.drop3 = nn.Dropout2d()
        self.pool3=nn.MaxPool2d(2)
        conv_layers += [self.conv3, self.drop3, self.pool3,self.relu3,self.bn3]

        # fourth Convolution Block
        self.conv4 = nn.Conv2d(32, 64, kernel_size=3)
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        nn.init.kaiming_normal_(self.conv4.weight, a=0.1)
        self.conv4.bias.data.zero_()
        self.drop4 = nn.Dropout2d()
        self.pool4=nn.MaxPool2d(2)
        conv_layers += [self.conv4, self.drop4, self.pool4,self.relu4,self.bn4]

        # 5 Block
        self.conv5 = nn.Conv2d(64, 64, kernel_size=3)
        self.relu5 = nn.ReLU()
        self.bn5 = nn.BatchNorm2d(64)
        nn.init.kaiming_normal_(self.conv5.weight, a=0.1)
        self.conv5.bias.data.zero_()
        self.drop5 = nn.Dropout2d()
        self.pool5=nn.MaxPool2d(2)
        conv_layers += [self.conv5, self.drop5, self.pool5,self.relu5,self.bn5]

        # Block 6
        self.conv6 = nn.Conv2d(64, 64, kernel_size=3, stride=(2, 2))
        self.relu6 = nn.ReLU()
        self.bn6 = nn.BatchNorm2d(64)
        nn.init.kaiming_normal_(self.conv6.weight, a=0.1)
        self.conv6.bias.data.zero_()
        self.drop6 = nn.Dropout2d()
        self.pool6=nn.MaxPool2d(2)
        conv_layers += [self.conv6, self.drop6, self.pool6,self.relu6,self.bn6]

        self.flatten=nn.Flatten()
        # Linear Classifier
        self.lin1 = nn.Linear(19264,50)
        self.lin2=nn.Linear(50,2)

        # Wrap the Convolutional Blocks
        self.conv = nn.Sequential(*conv_layers)
 
    # ----------------------------
    # Forward pass computations
    # ----------------------------
    def forward(self, x):
        # Run the convolutional blocks
        x = self.conv(x)

        # Adaptive pool and flatten for input to linear layer
        x = x.view(x.shape[0], -1)
        x=self.flatten(x)

        # Linear layer
        x = F.relu(self.lin1(x))
        x=self.lin2(x)

        # Final output
        x=F.log_softmax(x,dim=1)
        return x

# Create the model and put it on the GPU if available
myModel = CNNet()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = myModel.to(device)
# Check that it is on Cuda
next(myModel.parameters()).device

summary(myModel, input_size=(25,1,2813,512))

In [ ]:
# cost function used to determine best parameters
cost = torch.nn.CrossEntropyLoss()

# used to create optimal parameters
learning_rate = 0.0001
optimizer = torch.optim.Adam(myModel.parameters(), lr=learning_rate)

# Create the training function

def train(dataloader, model, loss, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, Y) in enumerate(dataloader):
        
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = cost(pred, Y)
        loss.backward()
        optimizer.step()

        if batch % 60 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f}  [{current:>5d}/{size:>5d}]')


# Create the validation/test function

def test(dataloader, model):
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (X, Y) in enumerate(dataloader):
            X, Y = X.to(device), Y.to(device)
            pred = model(X)

            test_loss += cost(pred, Y).item()
            correct += (pred.argmax(1)==Y).type(torch.float).sum().item()

    test_loss /= len(dataloader)
    correct /= len(dataloader.dataset)

    print(f'\nTest Error:\nacc: {(100*correct):>0.1f}%, avg loss: {test_loss:>8f}\n')

In [ ]:
#training
epochs = 15

for t in range(epochs):
    print(f'Epoch {t+1}\n-------------------------------')
    train(train_dataloader, myModel, cost, optimizer)
    test(test_dataloader, myModel)
print('Done!')

In [ ]:
#testing
myModel.eval()
test_loss, correct = 0, 0
reverse_class_map = ['bad', 'good']

test(train_dataloader,myModel)
test(test_dataloader,myModel)

        

In [ ]:
#predictions
def predict(data, model):
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for filename,(X, Y) in zip(data.imgs,data):
            X.unsqueeze_(-1)
            X=X.transpose(1,3)
            X=X.to(device)
            pred = model(X)
            pred=pred.argmax(1)
            print(filename,reverse_class_map[pred])
            


#convert to spectrogram first using data_prep
inference_dataset = datasets.ImageFolder(root='./data/test/',transform=transforms.Compose([transforms.Grayscale(),transforms.ToTensor()]))
predict(inference_dataset,myModel)
